# Homework 5

Due Monday, May 22, 7:59am.

1. Solve the machine allocation problem to minimize total time given that each machine has to complete the same number of tasks.
2. Suppose that it is OK for a machine to do slightly more tasks than another machine. In particular, for  the small data set, suppose we allow any machine to do at most 5 tasks, but it has to do at least 3 tasks. Solve the machine allocation problem to minimize total time.
3. Suppose that machine 0 and machine 1 cannot do odd numbered tasks.  Solve the machine allocation problem to minimize total time.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np

In [3]:
path_to_data = "/content/drive/MyDrive/BDM /machine_scheduling_small.csv"
machine_times = np.genfromtxt(path_to_data, delimiter=',')

In [4]:
import sys
import os
 
if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 MB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.1/294.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 10.5 MB/s eta 0:00:00
  Created wheel for bunch: filename=bunch-1.0.1-py3-none-any.whl size=7076 sha256=dbfbf7e2de0663d9e1a2790cdf960462cc99419b438af7929a87

In [5]:
machine_times

array([[39., 31., 28., 37., 40., 38., 26., 35., 28., 23., 24., 31., 27.,
        23., 25., 25., 23., 31., 27., 32.],
       [27., 30., 31., 24., 21., 24., 30., 35., 27., 30., 36., 21., 38.,
        31., 27., 24., 23., 28., 35., 32.],
       [36., 23., 34., 40., 23., 31., 30., 36., 38., 32., 39., 34., 20.,
        35., 40., 24., 26., 34., 38., 33.],
       [40., 34., 40., 36., 30., 32., 39., 40., 24., 39., 25., 37., 39.,
        40., 30., 29., 29., 21., 34., 35.],
       [29., 29., 28., 40., 29., 29., 23., 28., 29., 35., 21., 28., 24.,
        22., 28., 22., 30., 21., 34., 36.]])

In [6]:
number_of_machines = machine_times.shape[0]
number_of_tasks = machine_times.shape[1]
n_tasks_per_machine = int(number_of_tasks/number_of_machines)
print("Machines: ", number_of_machines)
print("Tasks: ", number_of_tasks)
print("Per machine: ", n_tasks_per_machine)

Machines:  5
Tasks:  20
Per machine:  4


# Problem 1

In [7]:
model = ConcreteModel()

# Define indices and parameters
number_of_machines = machine_times.shape[0]
number_of_tasks = machine_times.shape[1]
n_tasks_per_machine = int(number_of_tasks / number_of_machines)

model.Machines = RangeSet(1, number_of_machines)
model.Tasks = RangeSet(1, number_of_tasks)

# Define binary decision variables
model.x = Var(model.Machines, model.Tasks, within=Binary)

# Define the objective function
objective_expr = sum(model.x[m, t] * machine_times[m - 1, t - 1] for m in model.Machines for t in model.Tasks)
model.total_time = Objective(expr=objective_expr, sense=minimize)

# Define the constraint for the number of tasks per machine
model.tasks_per_machine = ConstraintList()
for m in model.Machines:
    tasks_expr = sum(model.x[m, t] for t in model.Tasks)
    model.tasks_per_machine.add(tasks_expr == n_tasks_per_machine)

# Solve the optimization model
SolverFactory('cbc').solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Feb  3 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmpwldzmytw.pyomo.lp -stat=1 -solve -solu /tmp/tmpwldzmytw.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 5 (-1) rows, 100 (-1) columns and 100 (-1) elements
Statistics for presolved model
Original problem has 100 integers (100 of which binary)
Presolved problem has 100 integers (100 of which binary)
==== 0 zero objective 21 different
==== absolute objective values 21 different
==== for integers 0 zero objective 21 different
==== for integers absolute objective values 21 different
===== end objective counts


Problem has 5 rows, 100 columns (100 with objective) and 100 elements
There are 100 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 100 of type 0.0

{'Problem': [{'Name': 'unknown', 'Lower bound': 457.0, 'Upper bound': 457.0, 'Number of objectives': 1, 'Number of constraints': 5, 'Number of variables': 100, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of nonzeros': 100, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.0858461856842041}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
# Print the optimal machine-task allocation
for m in model.Machines:
    for t in model.Tasks:
        if model.x[m, t]() == 1:
            print(f"Task {t} assigned to Machine {m}")

Task 10 assigned to Machine 1
Task 11 assigned to Machine 1
Task 14 assigned to Machine 1
Task 17 assigned to Machine 1
Task 4 assigned to Machine 2
Task 5 assigned to Machine 2
Task 12 assigned to Machine 2
Task 17 assigned to Machine 2
Task 2 assigned to Machine 3
Task 5 assigned to Machine 3
Task 13 assigned to Machine 3
Task 16 assigned to Machine 3
Task 9 assigned to Machine 4
Task 11 assigned to Machine 4
Task 16 assigned to Machine 4
Task 18 assigned to Machine 4
Task 11 assigned to Machine 5
Task 14 assigned to Machine 5
Task 16 assigned to Machine 5
Task 18 assigned to Machine 5


# Problem 2

In [9]:

model = ConcreteModel()

# Define indices and parameters
number_of_machines = machine_times.shape[0]
number_of_tasks = machine_times.shape[1]

model.Machines = RangeSet(1, number_of_machines)
model.Tasks = RangeSet(1, number_of_tasks)

# Define binary decision variables
model.x = Var(model.Machines, model.Tasks, within=Binary)

# Define the objective function
objective_expr = sum(model.x[m, t] * machine_times[m - 1, t - 1] for m in model.Machines for t in model.Tasks)
model.total_time = Objective(expr=objective_expr, sense=minimize)

# Define the constraint for the number of tasks per machine
model.min_tasks_per_machine = ConstraintList()
model.max_tasks_per_machine = ConstraintList()

min_tasks_per_machine_limit = 3
max_tasks_per_machine_limit = 5

for m in model.Machines:
    tasks_expr = sum(model.x[m, t] for t in model.Tasks)
    model.min_tasks_per_machine.add(tasks_expr >= min_tasks_per_machine_limit)
    model.max_tasks_per_machine.add(tasks_expr <= max_tasks_per_machine_limit)

# Solve the optimization model
SolverFactory('cbc').solve(model, tee=True)



Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Feb  3 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmpcs_59157.pyomo.lp -stat=1 -solve -solu /tmp/tmpcs_59157.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 10 (-1) rows, 100 (-1) columns and 200 (-1) elements
Statistics for presolved model
Original problem has 100 integers (100 of which binary)
Presolved problem has 100 integers (100 of which binary)
==== 0 zero objective 21 different
==== absolute objective values 21 different
==== for integers 0 zero objective 21 different
==== for integers absolute objective values 21 different
===== end objective counts


Problem has 10 rows, 100 columns (100 with objective) and 200 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 100 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 

{'Problem': [{'Name': 'unknown', 'Lower bound': 334.0, 'Upper bound': 334.0, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 100, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of nonzeros': 100, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.02574467658996582}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
# Print the optimal machine-task allocation
for m in model.Machines:
    for t in model.Tasks:
        if model.x[m, t]() == 1:
            print(f"Task {t} assigned to Machine {m}")


Task 10 assigned to Machine 1
Task 14 assigned to Machine 1
Task 17 assigned to Machine 1
Task 5 assigned to Machine 2
Task 12 assigned to Machine 2
Task 17 assigned to Machine 2
Task 2 assigned to Machine 3
Task 5 assigned to Machine 3
Task 13 assigned to Machine 3
Task 9 assigned to Machine 4
Task 11 assigned to Machine 4
Task 18 assigned to Machine 4
Task 11 assigned to Machine 5
Task 14 assigned to Machine 5
Task 18 assigned to Machine 5


# problem 3

In [11]:

model = ConcreteModel()

# Define indices and parameters
number_of_machines = machine_times.shape[0]
number_of_tasks = machine_times.shape[1]

model.Machines = RangeSet(1, number_of_machines)
model.Tasks = RangeSet(1, number_of_tasks)

# Define binary decision variables
model.x = Var(model.Machines, model.Tasks, within=Binary)

# Define the objective function
objective_expr = sum(model.x[m, t] * machine_times[m - 1, t - 1] for m in model.Machines for t in model.Tasks)
model.total_time = Objective(expr=objective_expr, sense=minimize)

# Define the constraint for the number of tasks per machine
model.min_tasks_per_machine = ConstraintList()
model.max_tasks_per_machine = ConstraintList()

min_tasks_per_machine_limit = 3
max_tasks_per_machine_limit = 5

for m in model.Machines:
    tasks_expr = sum(model.x[m, t] for t in model.Tasks)
    model.min_tasks_per_machine.add(tasks_expr >= min_tasks_per_machine_limit)
    model.max_tasks_per_machine.add(tasks_expr <= max_tasks_per_machine_limit)

# Define the constraint for machines 0 and 1 not doing odd-numbered tasks
model.odd_tasks_constraint = ConstraintList()
for m in [0, 1]:
    for t in model.Tasks:
        if t % 2 != 0:
            model.odd_tasks_constraint.add(model.x[m + 1, t] == 0)

# Solve the optimization model
SolverFactory('cbc').solve(model, tee=True)




Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Feb  3 2023 

command line - /content/bin/cbc -printingOptions all -import /tmp/tmp5u2x1698.pyomo.lp -stat=1 -solve -solu /tmp/tmp5u2x1698.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 10 (-21) rows, 80 (-21) columns and 160 (-61) elements
Statistics for presolved model
Original problem has 100 integers (100 of which binary)
Presolved problem has 80 integers (80 of which binary)
==== 0 zero objective 20 different
==== absolute objective values 20 different
==== for integers 0 zero objective 20 different
==== for integers absolute objective values 20 different
===== end objective counts


Problem has 10 rows, 80 columns (80 with objective) and 160 elements
Column breakdown:
0 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 80 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 o

{'Problem': [{'Name': 'unknown', 'Lower bound': 340.0, 'Upper bound': 340.0, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 80, 'Number of binary variables': 100, 'Number of integer variables': 100, 'Number of nonzeros': 80, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.01, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.025628328323364258}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
# Print the optimal machine-task allocation
for m in model.Machines:
    for t in model.Tasks:
        if model.x[m, t]() == 1:
            print(f"Task {t} assigned to Machine {m}")

Task 10 assigned to Machine 1
Task 14 assigned to Machine 1
Task 16 assigned to Machine 1
Task 4 assigned to Machine 2
Task 6 assigned to Machine 2
Task 12 assigned to Machine 2
Task 2 assigned to Machine 3
Task 5 assigned to Machine 3
Task 13 assigned to Machine 3
Task 9 assigned to Machine 4
Task 11 assigned to Machine 4
Task 18 assigned to Machine 4
Task 11 assigned to Machine 5
Task 14 assigned to Machine 5
Task 18 assigned to Machine 5
